# Advanced Prompt Engineering Techniques

# Advanced Prompt Engineering Techniques

Welcome to the hands-on exploration of advanced prompt engineering! In this notebook, you'll see these techniques in action and understand why prompt engineering is an **empirical science** that requires experimentation and iteration.

## What You'll Learn

- Core prompting techniques (zero-shot, few-shot, one-shot, role, emotional, chain-of-thought)
- Why LLM outputs are non-deterministic and how to control consistency
- Automatic prompt generation and meta-prompting strategies
- Token optimization techniques to reduce costs
- Model biases and how they affect outputs
- Using LLMs as judges to evaluate responses

## Prerequisites

Before starting, make sure you have:
- OpenAI API key set up
- Basic understanding of prompt engineering fundamentals (see lesson m2_01)
- Python environment with openai library installed

Let's dive in!

## 1. Setup & Configuration

First, let's import the necessary libraries and configure our OpenAI client.

In [ ]:
# Import required libraries
import os
from openai import OpenAI
import json
from collections import Counter
import time
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Set default model
MODEL = "gpt-4o-mini"

print("✅ Setup complete! OpenAI client initialized.")

### Helper Functions

Let's create some utility functions to make our experiments easier to manage and visualize.

In [ ]:
def call_openai(prompt, system_message="You are a helpful assistant.", temperature=0.7, max_tokens=None, seed=None):
    """Helper function to call OpenAI API with specified parameters"""
    try:
        params = {
            "model": MODEL,
            "messages": [
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature
        }
        
        if max_tokens:
            params["max_tokens"] = max_tokens
        if seed is not None:
            params["seed"] = seed
            
        response = client.chat.completions.create(**params)
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"

def display_response(title, response, params=None):
    """Display a response with formatting"""
    print(f"\n{'='*60}")
    print(f"📝 {title}")
    if params:
        print(f"Parameters: {params}")
    print(f"{'='*60}")
    print(response)
    print(f"{'='*60}\n")

def count_tokens_approx(text):
    """Approximate token count (roughly 4 characters per token)"""
    return len(text) // 4

print("✅ Helper functions loaded!")

## 2. Core Prompting Techniques

Let's explore the fundamental prompting techniques you learned about in the lesson. Each example demonstrates when and how to use these techniques effectively.

### Zero-Shot Prompting

Zero-shot prompting provides a clear, detailed description of what you need **without examples**. This is the most commonly used prompting technique in practice.

In [ ]:
# Zero-shot: Just a clear instruction
prompt = "Write a haiku about AI Consulting and Integration"

response = call_openai(prompt, temperature=0.7)
display_response("Zero-Shot Prompting", response)

### One-Shot Prompting

One-shot prompting provides **a single example** to demonstrate the expected output format. This helps the model understand the pattern you want.

In [ ]:
"""Q: What is prompt engineering?
A: Prompt engineering is the practice of crafting effective input prompts to guide AI models toward desired outputs or behaviors.

Q: Why is prompt engineering useful?
A:
"""

response = call_openai(prompt, temperature=0.7)
display_response("One-Shot Prompting", response)

### Few-Shot Prompting

Few-shot prompting adds **multiple examples** of the desired output format to guide the model's response, helping it understand the pattern more clearly.

In [ ]:
# Few-shot: Provide multiple examples
prompt = """Q: What is prompt engineering?
A: Prompt engineering is the practice of crafting effective input prompts to guide AI models toward desired outputs or behaviors.

Q: Why is prompt engineering useful?
A: Prompt engineering optimizes AI model outputs, enhancing accuracy, relevance, and task-specific performance.

Q: What is temperature in AI models?
A: Temperature controls the randomness of AI outputs, with lower values producing consistent results and higher values generating creative variations.

Q: What is chain-of-thought prompting?
A:"""

response = call_openai(prompt, temperature=0.7)
display_response("Few-Shot Prompting", response)

### Role Prompting

Role prompting asks the model to play a specific role or persona. Research shows it's more effective to define the assistant in **third person** ("they are a senior data scientist") rather than first person ("you are a senior data scientist").

In [ ]:
# Role prompting: Third person (more effective)
system_message = "The assistant is a senior data scientist with 15 years of experience in machine learning and statistical analysis."
prompt = "Explain the bias-variance tradeoff in simple terms."

response = call_openai(prompt, system_message=system_message, temperature=0.7)
display_response("Role Prompting (Third Person)", response, {"role": "Senior Data Scientist"})

### Emotional Prompting

Emotional prompting uses emotional stimuli to help the LLM better handle emotion and tone in its responses.

In [ ]:
# Emotional prompting: Adding emotional context
prompt = """This is very important to my career. I need you to explain AI ethics in a way that will help me 
succeed in my upcoming presentation. Please be thorough and thoughtful."""

response = call_openai(prompt, temperature=0.7)
display_response("Emotional Prompting", response)

### Chain-of-Thought Prompting

Chain-of-thought provides **step-by-step reasoning instructions**, enabling models to tackle complex tasks through structured thinking. This technique gave origin to reasoning models.

In [ ]:
# Chain-of-thought: Request step-by-step reasoning
prompt = """A store has 23 apples. They receive a shipment of 47 more apples, but 15 are bruised and must be discarded. 
They sell 38 apples. How many apples do they have left? 

Please solve this step-by-step, showing your reasoning at each stage."""

response = call_openai(prompt, temperature=0)
display_response("Chain-of-Thought Prompting", response)

## 3. Non-Deterministic Behavior Study

One of the most important concepts in prompt engineering is understanding that **LLM outputs are non-deterministic**. This means that running the same prompt multiple times can produce different results, even with identical parameters.

Let's explore this empirically!

### Part A: Demonstrating Inconsistency

We'll run the exact same prompt with identical parameters **10 times** to show how the outputs vary. This demonstrates why prompt engineering requires testing and iteration.

In [ ]:
# Run the same prompt 10 times with temperature=0.7
prompt = "Write a haiku about AI"
temperature = 0.7
num_runs = 10

print(f"Running the same prompt {num_runs} times with temperature={temperature}")
print(f"Prompt: '{prompt}'\n")
print("="*80)

responses = []
for i in range(num_runs):
    response = call_openai(prompt, temperature=temperature)
    responses.append(response)
    print(f"\n🔄 Run #{i+1}:")
    print(response)
    print("-"*80)
    time.sleep(0.5)  # Small delay to avoid rate limiting

print("\n" + "="*80)
print("📊 Analysis:")
print(f"Total unique responses: {len(set(responses))} out of {num_runs}")
print(f"Average length: {sum(len(r) for r in responses) / len(responses):.0f} characters")
print("\n💡 Key Insight: Even with the same prompt and parameters, we get different outputs!")
print("This is why prompt engineering is an empirical science - you must test and iterate.")

### Part B: Achieving Consistency

Now let's explore two ways to get **consistent and reproducible** results:
1. **Temperature = 0**: Produces the most deterministic outputs
2. **Seed parameter**: Ensures reproducible results across runs

In [ ]:
# Method 1: Temperature = 0 for consistency
print("Method 1: Using temperature=0 for maximum consistency")
print("="*80)

prompt = "Write a haiku about AI"
responses_temp0 = []

for i in range(5):
    response = call_openai(prompt, temperature=0)
    responses_temp0.append(response)
    print(f"\nRun #{i+1}: {response}")
    time.sleep(0.5)

print("\n" + "="*80)
print(f"📊 Unique responses with temperature=0: {len(set(responses_temp0))} out of 5")
print("💡 Temperature=0 produces highly consistent (though not always identical) results!")

In [ ]:
# Method 2: Using seed for reproducibility
print("\nMethod 2: Using seed parameter for reproducibility")
print("="*80)

prompt = "Write a haiku about AI"
seed_value = 42
responses_seeded = []

for i in range(5):
    response = call_openai(prompt, temperature=0.7, seed=seed_value)
    responses_seeded.append(response)
    print(f"\nRun #{i+1}: {response}")
    time.sleep(0.5)

print("\n" + "="*80)
print(f"📊 Unique responses with seed={seed_value}: {len(set(responses_seeded))} out of 5")
print("💡 Using a seed parameter gives you reproducible results!")

### When to Use Consistency vs. Variation

**Use temperature=0 or seed when:**
- Testing and debugging prompts
- You need reproducible results for comparison
- Consistency is critical (e.g., data extraction, classification)

**Use higher temperature when:**
- You want creative, diverse outputs
- Generating multiple variations (e.g., marketing copy, brainstorming)
- The task benefits from variety

## 4. Automatic Prompt Generation

One powerful technique is to use AI to **generate and refine prompts** for you. This is called meta-prompting or automatic prompt generation. Let's explore three practical examples.

### Example 1: Meta-Prompting - Improving a Basic Prompt

Let's start with a weak prompt and use GPT to generate an improved version.

In [ ]:
# Start with a weak prompt
weak_prompt = "Explain machine learning"

print("Original weak prompt:")
print(f"'{weak_prompt}'\n")

# Test the weak prompt
weak_response = call_openai(weak_prompt, temperature=0)
display_response("Response from Weak Prompt", weak_response)

In [ ]:
# Use meta-prompting to improve it
meta_prompt = f"""I have this prompt: "{weak_prompt}"

Please improve this prompt to make it more effective by:
1. Adding specific context about the target audience
2. Specifying the desired format and length
3. Including relevant constraints or requirements
4. Making the output more actionable

Return ONLY the improved prompt, without explanation."""

improved_prompt = call_openai(meta_prompt, temperature=0.7)
print("\n" + "="*80)
print("🚀 Improved Prompt Generated:")
print("="*80)
print(improved_prompt)
print("="*80)

In [ ]:
# Test the improved prompt
improved_response = call_openai(improved_prompt, temperature=0)
display_response("Response from Improved Prompt", improved_response)

print("\n📊 Comparison:")
print(f"Weak prompt length: {len(weak_prompt)} characters")
print(f"Improved prompt length: {len(improved_prompt)} characters")
print(f"Weak response length: {len(weak_response)} characters")
print(f"Improved response length: {len(improved_response)} characters")
print("\n💡 Notice how the improved prompt produces more structured, specific output!")

### Example 2: Task-Specific Prompt Generation

Let's define a specific task and have the AI generate multiple prompt variations automatically.

In [ ]:
# Define a task
task_description = """Task: Extract key information from customer feedback emails including:
- Customer sentiment (positive/negative/neutral)
- Main issue or topic
- Urgency level (low/medium/high)
- Suggested action"""

# Generate multiple prompt variations
generation_prompt = f"""{task_description}

Generate 3 different prompt variations for this task:
1. A concise, direct prompt
2. A detailed prompt with examples
3. A prompt that uses role-playing

Format each prompt clearly with numbers."""

prompt_variations = call_openai(generation_prompt, temperature=0.8)
print("🎯 Generated Prompt Variations:")
print("="*80)
print(prompt_variations)
print("="*80)

In [ ]:
# Test one of the variations with sample data
sample_email = """Subject: Urgent - System keeps crashing!

Hi, I'm really frustrated. Your software has crashed 3 times today and I've lost my work each time. 
This is completely unacceptable for a paid product. I need this fixed ASAP or I want a refund.

- John"""

# Use the first variation (adjust based on output)
test_prompt = f"""Analyze this customer feedback and extract:
- Sentiment
- Main issue
- Urgency
- Suggested action

Email: {sample_email}"""

analysis = call_openai(test_prompt, temperature=0)
display_response("Customer Feedback Analysis", analysis)

print("💡 Auto-generated prompts can help you quickly find the best approach for your task!")

### Example 3: Iterative Refinement

Create a feedback loop where the model refines its own prompt based on output quality.

In [ ]:
# Initial prompt for a task
current_prompt = "Write a product description for wireless earbuds"
iterations = 3

print("🔄 Iterative Prompt Refinement")
print("="*80)

for i in range(iterations):
    print(f"\n### Iteration {i+1} ###")
    print(f"Current prompt: {current_prompt}")
    
    # Generate output with current prompt
    output = call_openai(current_prompt, temperature=0.7)
    print(f"\nOutput:\n{output}")
    
    if i < iterations - 1:  # Don't refine on last iteration
        # Ask model to refine the prompt based on the output
        refinement_prompt = f"""I used this prompt: "{current_prompt}"
        
And got this output: "{output}"

The output is good but could be better. Improve the prompt to generate:
- More specific technical details
- Stronger emotional appeal
- Clear call-to-action

Return ONLY the improved prompt."""
        
        current_prompt = call_openai(refinement_prompt, temperature=0.7)
        print(f"\n✨ Refined prompt: {current_prompt}")
        print("-"*80)
        time.sleep(0.5)

print("\n" + "="*80)
print("💡 Iterative refinement helps you converge on the optimal prompt for your needs!")

## 5. Token Optimization

Tokens are the basic units that AI models process. **Every token costs money**, so optimizing your prompts to use fewer tokens while maintaining quality can significantly reduce costs.

💰 As a rough approximation: 1 token ≈ 4 characters (or 0.75 words)

### Example 1: Before/After Prompt Optimization

Let's take a verbose prompt and optimize it for token efficiency.

In [ ]:
# Verbose prompt (before optimization)
verbose_prompt = """Hello! I was wondering if you could possibly help me understand something. 
I'm trying to learn about artificial intelligence and machine learning, and I'm particularly 
interested in understanding what neural networks are and how they actually work. Could you 
please explain this to me in a way that would be easy for a beginner to understand? 
I would really appreciate it if you could provide a clear and simple explanation. Thank you!"""

# Optimized prompt (after removing fluff)
optimized_prompt = "Explain neural networks for beginners."

print("VERBOSE PROMPT:")
print(verbose_prompt)
print(f"\n📊 Approximate tokens: {count_tokens_approx(verbose_prompt)}")
print(f"Characters: {len(verbose_prompt)}")

print("\n" + "="*80)
print("\nOPTIMIZED PROMPT:")
print(optimized_prompt)
print(f"\n📊 Approximate tokens: {count_tokens_approx(optimized_prompt)}")
print(f"Characters: {len(optimized_prompt)}")

print("\n" + "="*80)
print(f"💰 Token reduction: {count_tokens_approx(verbose_prompt) - count_tokens_approx(optimized_prompt)} tokens saved!")
print(f"📉 Efficiency gain: {((count_tokens_approx(verbose_prompt) - count_tokens_approx(optimized_prompt)) / count_tokens_approx(verbose_prompt) * 100):.1f}% reduction")

In [ ]:
# Test both prompts to compare output quality
print("Testing verbose prompt...")
verbose_response = call_openai(verbose_prompt, temperature=0)
display_response("Verbose Prompt Response", verbose_response)

print("Testing optimized prompt...")
optimized_response = call_openai(optimized_prompt, temperature=0)
display_response("Optimized Prompt Response", optimized_response)

print("\n💡 Notice: The optimized prompt produces similar quality output with 85% fewer tokens!")

### Example 2: Systematic Token Reduction Strategies

Here are specific techniques to reduce token usage while maintaining clarity.

In [ ]:
# Strategy 1: Remove filler words
before_1 = "Could you please help me understand how AI works?"
after_1 = "Explain how AI works."

# Strategy 2: Use abbreviations
before_2 = "Explain artificial intelligence and machine learning"
after_2 = "Explain AI and ML"

# Strategy 3: Use imperatives instead of questions
before_3 = "Can you tell me what are the benefits of cloud computing?"
after_3 = "List cloud computing benefits."

# Strategy 4: Remove redundancy
before_4 = "Please provide a detailed and comprehensive explanation of neural networks"
after_4 = "Explain neural networks comprehensively."

strategies = [
    ("Remove filler words", before_1, after_1),
    ("Use abbreviations", before_2, after_2),
    ("Use imperatives", before_3, after_3),
    ("Remove redundancy", before_4, after_4)
]

print("📋 Token Optimization Strategies:\n")
total_saved = 0

for strategy, before, after in strategies:
    before_tokens = count_tokens_approx(before)
    after_tokens = count_tokens_approx(after)
    saved = before_tokens - after_tokens
    total_saved += saved
    
    print(f"\n{strategy}:")
    print(f"  Before: \"{before}\" ({before_tokens} tokens)")
    print(f"  After:  \"{after}\" ({after_tokens} tokens)")
    print(f"  💰 Saved: {saved} tokens ({(saved/before_tokens*100):.0f}% reduction)")

print(f"\n{'='*80}")
print(f"📊 Total tokens saved across examples: {total_saved}")
print("\n💡 Apply these strategies consistently to reduce costs significantly!")

### Example 3: Calculate Cost Savings

Let's calculate the actual cost difference using OpenAI's pricing (as of 2024).

In [ ]:
# OpenAI GPT-4o-mini pricing (example - check current pricing)
INPUT_COST_PER_1K = 0.00015  # $0.15 per 1M tokens
OUTPUT_COST_PER_1K = 0.0006   # $0.60 per 1M tokens

def calculate_cost(input_tokens, output_tokens):
    """Calculate cost based on token usage"""
    input_cost = (input_tokens / 1000) * INPUT_COST_PER_1K
    output_cost = (output_tokens / 1000) * OUTPUT_COST_PER_1K
    return input_cost + output_cost

# Example scenario: Running 1000 queries per day
queries_per_day = 1000
days_per_month = 30

# Scenario A: Verbose prompts
verbose_input_tokens = 150
verbose_output_tokens = 200

# Scenario B: Optimized prompts (50% reduction)
optimized_input_tokens = 75
optimized_output_tokens = 200  # Output stays same

# Calculate costs
verbose_cost_per_query = calculate_cost(verbose_input_tokens, verbose_output_tokens)
optimized_cost_per_query = calculate_cost(optimized_input_tokens, optimized_output_tokens)

verbose_monthly_cost = verbose_cost_per_query * queries_per_day * days_per_month
optimized_monthly_cost = optimized_cost_per_query * queries_per_day * days_per_month

savings_per_month = verbose_monthly_cost - optimized_monthly_cost
savings_per_year = savings_per_month * 12

print("💰 COST ANALYSIS")
print("="*80)
print(f"\nScenario: {queries_per_day} queries/day, {days_per_month} days/month")
print(f"\nVerbose Prompts:")
print(f"  - Cost per query: ${verbose_cost_per_query:.6f}")
print(f"  - Monthly cost: ${verbose_monthly_cost:.2f}")
print(f"\nOptimized Prompts:")
print(f"  - Cost per query: ${optimized_cost_per_query:.6f}")
print(f"  - Monthly cost: ${optimized_monthly_cost:.2f}")
print(f"\n{'='*80}")
print(f"💵 SAVINGS:")
print(f"  - Per month: ${savings_per_month:.2f}")
print(f"  - Per year: ${savings_per_year:.2f}")
print(f"  - Percentage saved: {(savings_per_month/verbose_monthly_cost*100):.1f}%")
print(f"\n{'='*80}")
print("💡 Token optimization isn't just about efficiency—it directly impacts your budget!")

## 6. LLM as Judge (Brief Example)

LLMs can evaluate the outputs from other LLMs. This technique is called "LLM as Judge" and is useful for comparing different prompts or responses.

**Note:** This will be covered extensively in weeks 3, 4, and 6. Here's a quick preview.

In [ ]:
# Generate multiple responses to evaluate
prompt = "Explain quantum computing in simple terms."

responses_to_judge = []
for i in range(3):
    response = call_openai(prompt, temperature=0.8)
    responses_to_judge.append(response)
    print(f"\nResponse {i+1}:")
    print(response)
    print("-"*80)
    time.sleep(0.5)

In [ ]:
# Use LLM as judge to evaluate and rank them
judge_prompt = f"""Evaluate these 3 explanations of quantum computing. Rate each on:
1. Clarity (1-10)
2. Accuracy (1-10)
3. Accessibility for beginners (1-10)

Response 1: {responses_to_judge[0]}

Response 2: {responses_to_judge[1]}

Response 3: {responses_to_judge[2]}

Provide scores for each response and recommend the best one. Format your response clearly."""

judgment = call_openai(judge_prompt, temperature=0)
display_response("LLM Judge Evaluation", judgment)

print("\n💡 Note: LLM as Judge will be covered in detail in weeks 3, 4, and 6!")
print("You'll learn advanced evaluation techniques and best practices.")

## 7. Model Biases Demonstration

AI models exhibit various cognitive biases that affect their outputs. Understanding these biases helps you design better prompts to mitigate them.

### Recency Bias

Models tend to remember what's at the **end** of the prompt and forget what's at the **beginning**.

In [ ]:
# Test recency bias: important info at beginning vs end
prompt_important_first = """IMPORTANT: Your response must be exactly 2 sentences.

Explain the greenhouse effect and its impact on climate change."""

prompt_important_last = """Explain the greenhouse effect and its impact on climate change.

IMPORTANT: Your response must be exactly 2 sentences."""

print("Testing Recency Bias...\n")
print("="*80)

response_first = call_openai(prompt_important_first, temperature=0.7)
response_last = call_openai(prompt_important_last, temperature=0.7)

print("Important constraint at BEGINNING:")
print(response_first)
sentences_first = response_first.count('.') 
print(f"Sentences: {sentences_first}\n")

print("-"*80)
print("\nImportant constraint at END:")
print(response_last)
sentences_last = response_last.count('.')
print(f"Sentences: {sentences_last}\n")

print("="*80)
print("💡 The model often follows the constraint better when it's at the END!")

### Verbosity Bias

Models tend to generate **long, elaborate responses** rather than brief, concise answers.

In [ ]:
# Test verbosity bias
prompt_brief = "What is machine learning?"
prompt_explicit_brief = "What is machine learning? Answer in one sentence."

print("Testing Verbosity Bias...\n")
print("="*80)

response_natural = call_openai(prompt_brief, temperature=0.7)
response_constrained = call_openai(prompt_explicit_brief, temperature=0.7)

print("Without explicit brevity instruction:")
print(response_natural)
print(f"\nLength: {len(response_natural)} characters\n")

print("-"*80)
print("\nWith explicit brevity instruction:")
print(response_constrained)
print(f"\nLength: {len(response_constrained)} characters\n")

print("="*80)
print("💡 Without constraints, models tend toward verbose responses!")

### List-Making Bias

Models tend to format responses as **bullet points or numbered lists** rather than natural prose paragraphs.

In [ ]:
# Test list-making bias
prompt_neutral = "What are the benefits of exercise?"
prompt_prose = "Write a paragraph about the benefits of exercise. Use flowing prose, not lists."

print("Testing List-Making Bias...\n")
print("="*80)

response_neutral = call_openai(prompt_neutral, temperature=0.7)
response_prose = call_openai(prompt_prose, temperature=0.7)

print("Without format specification:")
print(response_neutral)
print(f"\nContains bullet points or numbers: {('•' in response_neutral or '1.' in response_neutral or '-' in response_neutral)}")

print("\n" + "-"*80)
print("\nWith prose instruction:")
print(response_prose)
print(f"\nContains bullet points or numbers: {('•' in response_prose or '1.' in response_prose or '-' in response_prose[:50])}")

print("\n" + "="*80)
print("💡 Models default to lists—explicitly request prose format if needed!")

## 8. Summary & Best Practices

Congratulations! You've explored advanced prompt engineering techniques. Let's recap the key takeaways.

### Key Takeaways

**1. Core Techniques:**
- Zero-shot: Clear descriptions without examples (most common)
- One/Few-shot: Provide examples to guide format and style
- Role prompting: Define assistant in third person for better results
- Emotional prompting: Use emotion to influence tone
- Chain-of-thought: Request step-by-step reasoning for complex tasks

**2. Non-Deterministic Nature:**
- LLM outputs vary between runs with same parameters
- Use temperature=0 for consistency
- Use seed parameter for reproducibility
- Test prompts multiple times before deployment

**3. Automatic Prompt Generation:**
- Use AI to improve your prompts (meta-prompting)
- Generate multiple variations automatically
- Implement iterative refinement loops
- Save time and discover better approaches

**4. Token Optimization:**
- Remove filler words and redundancy
- Use abbreviations and imperatives
- Token reduction = cost reduction
- Maintain quality while minimizing tokens

**5. Model Biases:**
- Recency bias: Put important info at the end
- Verbosity bias: Explicitly request brevity
- List-making bias: Request prose format when needed
- Design prompts to mitigate known biases

**6. LLM as Judge:**
- Use LLMs to evaluate other LLM outputs
- Compare prompt variations objectively
- More advanced techniques coming in weeks 3, 4, and 6

### The Prompt Engineering Workflow

Remember: Prompt engineering is an **empirical science**.

```
1. CREATE → 2. TEST → 3. ITERATE
     ↑                       ↓
     └───────────────────────┘
```

**Create:** Write your initial prompt based on best practices

**Test:** Run it multiple times with your actual use case

**Iterate:** Refine based on results, biases, and failures

**Repeat:** Continue until you achieve consistent, quality outputs

### Building Your Prompt Toolkit

**Save successful prompts** as templates or presets:
- Data extraction prompts
- Classification prompts
- Creative writing prompts
- Analysis and summarization prompts

**Create templates** for common tasks:
- Include placeholders for variable content
- Document which parameters work best
- Note any special considerations or biases

**Version control** your prompts:
- Track iterations and improvements
- Document what works and what doesn't
- Share learnings with your team

### Quick Reference: Common Problems & Solutions

| Problem | Solution |
|---------|----------|
| **Hallucination** | Lower temperature, add examples |
| **Off-topic** | Stronger system message, lower top-p |
| **Too verbose** | Set max_tokens limit, use stop sequences |
| **Repetitive** | Increase repetition penalties |
| **Inconsistent** | Set temperature=0, use seed value |
| **Wrong format** | Enable JSON mode, provide clear format instructions |
| **Ignoring instructions** | Move key instructions to end (recency bias) |
| **Too expensive** | Optimize tokens, use cheaper models where appropriate |

### Next Steps

Now that you understand advanced prompt engineering:

1. **Practice** with the techniques in this notebook
2. **Apply** these strategies to your own projects
3. **Experiment** with different combinations
4. **Build** your own prompt library
5. **Look forward** to RAG and ReAct prompting in Week 3!

### Additional Resources

- OpenAI Prompt Engineering Guide: https://platform.openai.com/docs/guides/prompt-engineering
- Prompt Engineering Papers: https://github.com/dair-ai/Prompt-Engineering-Guide
- Community Prompts: https://prompts.chat

---

**Happy Prompting! 🚀**